In [ ]:
import spacy
import json
import random
import re
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn import model_selection
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.symbols import ORTH
from spacy.symbols import NORM
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
import matplotlib.pyplot as plt
import random
import math
import plotly.express as px
import sys 
sys.path.append('./luima_sbd')
import luima_sbd.sbd_utils as luima
from spacy.language import Language
random.seed(42)
from sklearn.utils import shuffle


In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          title=None,
                          cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
def top_tfidf_features(row, features, top_n=15):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df


def top_features_in_doc(Xtr, features, row_id, top_n=15):
    ''' Top tfidf features in specific document (matrix row) '''
    xtr_row = Xtr[row_id]
    if type(xtr_row) is not np.ndarray:
        xtr_row = xtr_row.toarray()
    row = np.squeeze(xtr_row)
    return top_tfidf_features(row, features, top_n)


def top_mean_features(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids]
    else:
        D = Xtr
    if type(D) is not np.ndarray:
        D = D.toarray()
    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_features(tfidf_means, features, top_n)


def top_features_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = {}
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_features(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs[label] = feats_df
    return dfs


def span_top_tfidf(spans_txt, spans_tfidf, features, index):
    print('span text:\n'+spans_txt[index]+'\n')
    print(top_features_in_doc(spans_tfidf, features, index))

In [ ]:
corpus_fpath = './ldsi_s2021/ldsi_bva_sentence_corpus_v1.json'
data = json.load(open(corpus_fpath))
affirmed = open('./ldsi_s2021/affirmed_ids.txt', 'r').read().split("\n")
denied= open('./ldsi_s2021/denied_ids.txt', 'r').read().split("\n")
remanded = open('./ldsi_s2021/remanded_ids.txt', 'r').read().split("\n")
# print(len(affirmed), len(denied), len(remanded))
annotations = data['annotations']
documents_by_id = {d['_id']: d for d in data['documents']}
types_by_id = {t['_id']: t for t in data['types']}
type_ids_by_name = {t['name']: t['_id'] for t in data['types']}
type_names_by_id = {t['_id']: t['name'] for t in data['types']}
doc_id_by_name = {d['name']: d['_id'] for d in data['documents']}
doc_name_by_id = {d['_id']: d['name'] for d in data['documents']}

def make_span_data(documents_by_id, types_by_id, annotations):
    span_data = []
    for a in annotations:
        start = a['start']
        end = a['end']
        document_txt = documents_by_id[a['document']]['plainText']
        atype = a['type']
        document_name=documents_by_id[a['document']]['name']
        if document_name in affirmed:
            decision='affirmed'
        elif document_name in denied:
            decision='denied'
        elif document_name in remanded:
            decision='remanded'
        sd = {'txt': document_txt[start:end],
              'document': a['document'],
              'type': types_by_id[atype]['name'],
              'start': a['start'],
              'start_normalized': a['start'] / len(document_txt),
              'end': a['end'],
              'name': document_name,
              'decisions': decision}
        span_data.append(sd)
    return span_data

spans = make_span_data(documents_by_id, types_by_id, annotations)
span_labels = [s['type'] for s in spans]
span_decisions = [s['decisions'] for s in spans]

In [ ]:
random.seed(42)
aff=random.sample(affirmed, 6)
den=random.sample(denied, 6)
rem=random.sample(remanded, 6)
test_affirm, dev_affirm = aff[0:3], aff[3:6] 
test_denied, dev_denied = den[0:3], den[3:6] 
test_remanded, dev_remanded = rem[0:3], rem[3:6] 

test_ids = test_affirm+test_denied+test_remanded
dev_ids = dev_affirm+dev_denied+dev_remanded

test_spans=[]
dev_spans=[]
train_spans=[]
for s in spans:
    if s['name'] in test_ids:
        test_spans.append(s)
    elif s['name'] in dev_ids:
        dev_spans.append(s)
    else:
        train_spans.append(s)
        
unique_files=pd.DataFrame(train_spans).name.unique()

In [ ]:
print(test_ids)
print(dev_ids)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
nlp.tokenizer.add_special_case('Vet. App.', [{ORTH: 'Vet. App.'}])
nlp.tokenizer.add_special_case('Fed. Cir.', [{ORTH: 'Fed. Cir.'}])

In [ ]:
def spacy_tokenize(txt):
    nlp.disable_pipes('parser')
    doc = nlp.pipe(txt, n_process=4)
    doc = nlp(txt)
    tokens = list(doc)
    clean_tokens = []
    for i in range(len(tokens)):
        t=tokens[i]
        t1=tokens[i]
#         print(t.pos_, t.text)
        if(i!=len(tokens)-1):
            t1=tokens[i+1]
        if(t1!=t and t1.pos_=='PART' and re.search(r'\'', t1.text)):
            scrap = t.text+t1.text
            scrap = re.sub(r'\W','',scrap).lower()
            clean_tokens.append(scrap)
            i=i+1           
        elif t.pos_ == 'PUNCT':
            pass
        elif t.text in ('Vet. App.','Fed. Cir.'):
            lem=t.lemma_
            lem=lem.lower()
            clean_tokens.append(lem)
        elif (t.text[0].isalpha()==False and t.is_digit==False):
            if(t.is_upper==False):
                pass
            else:
                lem=t.lemma_
                lem=lem.lower()
                clean_tokens.append(lem)            
        elif t.pos_ == 'NUM':
            clean_tokens.append(f'<NUM{len(t)}>')
        else:
            lem=t.lemma_
            lem = re.sub(r'\W','',lem)
            lem=lem.lower()
            clean_tokens.append(lem)
    return clean_tokens

def spans_add_spacy_tokens(spans):
    for s in spans:
        s['tokens_spacy'] = spacy_tokenize(s['txt'])

In [ ]:
spans_add_spacy_tokens(train_spans)
spans_add_spacy_tokens(test_spans)
spans_add_spacy_tokens(dev_spans)

In [ ]:
# 0600090.txt
df_segmented=pd.read_pickle("./sentence_segmented_dict.pkl")

In [ ]:
example_basic_1 = 'In sum, as the preponderance of the evidence is against the Veteran\'s claim, his appeal must be denied.'
example_cit_1 = 'Smith v. Gober, 14 Vet. App. 227 (2000), aff\'d 281 F.3d 1384 (Fed. Cir. 2002); Dela Cruz v. Principi, 15 Vet. App. 143 (2001); see also Quartuccio v. Principi, 16 Vet. App. 183 (2002).'
example_rule_1 = '"To establish a right to compensation for a present disability, a Veteran must show: "(1) the existence of a present disability; (2) in-service incurrence or aggravation of a disease or injury; and (3) a causal relationship between the present disability and the disease or injury incurred or aggravated during service"-the so-called "nexus" requirement."'
example_mixed_1 = 'In Dingess v. Nicholson, 19 Vet. App. 473 (2006), the U.S. Court of Appeals for Veterans Claims held that, upon receipt of an application for a service-connection claim, 38 U.S.C.A. � 5103(a) and 38 C.F.R. � 3.159(b) require VA to provide the claimant with notice that a disability rating and an effective date for the award of benefits will be assigned if service connection is awarded. '
debug = "The Veterans Claims Assistance Act of 2000 (VCAA), Pub. L. No. 106-475, 114 Stat. 2096 (Nov. 9, 2000) (codified at 38 U.S.C.A. §§ 5100, 5102, 5103, 5103A, 5106, 5107, and 5126 (West 2002) redefined VA's duty to assist the veteran in the development of a claim."
print("ONE", spacy_tokenize(example_basic_1))
print("TWO", spacy_tokenize(example_cit_1))
print("THREE", spacy_tokenize(example_rule_1))
print("FOUR", spacy_tokenize(example_mixed_1))
print("FIVE", spacy_tokenize(debug))

In [ ]:
token_track=[]
def func_tok(file, sentences):
    for s in sentences:
        tokens = spacy_tokenize(s)
        token_dict = {
            "file_name": file,
            "sentence": s,
            "tokens": tokens,
            "token_count": len(tokens)
        }
        token_track.append(token_dict)
        
%time df_segmented.apply(lambda x: func_tok(x.file_name, x.sentences), axis=1)
print("Done")

In [ ]:
pd.DataFrame(token_track).to_pickle("./token_df.pkl")

In [ ]:
df_test=pd.read_pickle("./token_df.pkl")

In [ ]:
df_test

In [ ]:
df_test = shuffle(df_test)

In [ ]:
file_object = open('fastTextFinalShuffled.txt', 'a')

def stringify(tokens):
    string=' '.join(tokens)
    string=string+'\n'
    file_object.write(string)

%time df_test[df_test.token_count>5].tokens.apply(stringify)

file_object.close()

In [ ]:
for sentence in df_test.sentence[df_test.sentence.str.contains("211.a")]:
        print("one", sentence)

In [ ]:
df_test

In [ ]:
bin_width= 4
# here you can choose your rounding method, I've chosen math.ceil
nbins = math.ceil((df_test["token_count"].max() - df_test["token_count"].min()) / bin_width)

fig = px.histogram(df_test, x="token_count", nbins=nbins,
            width=1200, height=600,
            labels={ # replaces default labels by column name
                "token_count": "Number of Tokens",
            },
            template="plotly_white")
fig.update_yaxes(showgrid=True)
fig.update_layout(title_text="Histogram of Number of Tokens in Each Sentence", title_x=0.5, font_size=18)
fig.show()

In [ ]:
df_test[df_test.token_count>5].shape

In [ ]:
df_test.shape